In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from utils import save_plot

In [ ]:
# Simple CNN (like before)
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv = nn.Sequential(
            nn.Conv2d(1, 32, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2),
            nn.Conv2d(32, 64, 3, padding=1), nn.ReLU(),
            nn.MaxPool2d(2)
        )
        self.fc = nn.Sequential(
            nn.Linear(64*7*7, 128), nn.ReLU(),
            nn.Linear(128, 10)
        )
    def forward(self, x):
        x = self.conv(x)
        x = x.view(x.size(0), -1)
        return self.fc(x)

In [ ]:
# Train loop (short training for comparison)
def train_model(optimizer_type="SGD", epochs=3, lr=0.01):
    # Load MNIST
    transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])
    trainset = torchvision.datasets.MNIST(root="./data", train=True, download=True, transform=transform)
    trainloader = torch.utils.data.DataLoader(trainset, batch_size=64, shuffle=True)

    device = "cuda" if torch.cuda.is_available() else "cpu"
    model = SimpleCNN().to(device)
    criterion = nn.CrossEntropyLoss()

    if optimizer_type == "SGD":
        optimizer = optim.SGD(model.parameters(), lr=lr)
    elif optimizer_type == "Adam":
        optimizer = optim.Adam(model.parameters(), lr=lr)
    elif optimizer_type == "RMSProp":
        optimizer = optim.RMSprop(model.parameters(), lr=lr)

    losses = []
    for epoch in range(epochs):
        running_loss = 0
        for inputs, labels in trainloader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        avg_loss = running_loss / len(trainloader)
        losses.append(avg_loss)
        print(f"{optimizer_type} Epoch {epoch+1}/{epochs}, Loss: {avg_loss:.4f}")
    return losses

In [ ]:
# Run comparison
loss_sgd = train_model("SGD", lr=0.01)
loss_adam = train_model("Adam", lr=0.001)
loss_rms = train_model("RMSProp", lr=0.001)

In [ ]:
# Plot
fig, ax = plt.subplots()
ax.plot(loss_sgd, label="SGD")
ax.plot(loss_adam, label="Adam")
ax.plot(loss_rms, label="RMSProp")
ax.set_title("Optimizer Comparison on MNIST")
ax.set_xlabel("Epochs")
ax.set_ylabel("Loss")
ax.legend()
save_plot(fig, "optimizer_comparison.png")